In [8]:
import requests
import json
import pandas as pd
from pprint import pprint

# Use Pandas to display tables nicely in the notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

print("Libraries successfully imported.")

Libraries successfully imported.


In [ ]:
# --- CONFIGURATION ---
CANVAS_DOMAIN = "canvas.asu.edu" # e.g., "canvas.asu.edu"
CANVAS_TOKEN = "YOUR TOKEN HERE"

# The ID of the course you want to explore (e.g., your Testing Ground course ID)
COURSE_ID = "240102"  # e.g., 240102

# Base URL for all API requests
API_BASE_URL = f"https://{CANVAS_DOMAIN}/api/v1"

# Authentication header required for all raw requests
HEADERS = {
    "Authorization": f"Bearer {CANVAS_TOKEN}"
}

print(f"Configured to connect to {CANVAS_DOMAIN} for Course {COURSE_ID}")

Configured to connect to canvas.asu.edu for Course 240102


In [10]:
def make_canvas_get_request(endpoint, params=None):
    """
    Makes a raw GET request to the Canvas API.
    Args:
        endpoint (str): The API endpoint (e.g., "/courses/1/assignments")
        params (dict): Optional query parameters.
    Returns:
        dict or list: The JSON response from Canvas, or None if failed.
    """
    url = f"{API_BASE_URL}/{endpoint}"
    print(f"-> Requesting: {url}")
    
    response = requests.get(url, headers=HEADERS, params=params)
    
    if response.status_code == 200:
        print("-> Success!")
        return response.json()
    else:
        print(f"-> Error! Status Code: {response.status_code}")
        print(f"-> Response: {response.text}")
        return None

print("Helper function defined.")

Helper function defined.


In [12]:
endpoint = f"courses/{COURSE_ID}"

# Make the request
raw_course_data = make_canvas_get_request(endpoint)

# View the raw JSON structure clearly
if raw_course_data:
    print("\n--- RAW COURSE DATA (Top 5 keys for brevity) ---")
    # Printing just the first few keys to avoid cluttering the notebook, 
    # but you can use pprint(raw_course_data) to see EVERYTHING.
    first_few_items = {k: raw_course_data[k] for k in list(raw_course_data)}
    pprint(first_few_items)

-> Requesting: https://canvas.asu.edu/api/v1/courses/240102
-> Success!

--- RAW COURSE DATA (Top 5 keys for brevity) ---
{'account_id': 63,
 'apply_assignment_group_weights': False,
 'blueprint': False,
 'calendar': {'ics': 'https://canvas.asu.edu/feeds/calendars/course_dByiwUGLJ0lgBMjgxnE7CHWe51a5vlZGBsp9hAC2.ics'},
 'course_code': 'TRN-2025Fall-sdosburn',
 'course_color': None,
 'created_at': '2025-10-09T19:35:40Z',
 'default_view': 'modules',
 'end_at': None,
 'enrollment_term_id': 504,
 'enrollments': [{'enrollment_state': 'active',
                  'limit_privileges_to_course_section': False,
                  'role': 'TaEnrollment',
                  'role_id': 5,
                  'type': 'ta',
                  'user_id': 721566}],
 'friendly_name': None,
 'grade_passback_setting': None,
 'grading_standard_id': 2300,
 'hide_final_grades': False,
 'homeroom_course': False,
 'id': 240102,
 'is_public': False,
 'is_public_to_auth_users': False,
 'license': None,
 'name': 'Testin

In [13]:
endpoint = f"courses/{COURSE_ID}/assignments"
params = {"per_page": 50}

raw_assignments_list = make_canvas_get_request(endpoint, params=params)

if raw_assignments_list:
    print(f"Found {len(raw_assignments_list)} assignments.")
    
    # OPTION 1: See the raw JSON of the first assignment to understand its structure
    print("\n--- RAW JSON OF FIRST ASSIGNMENT ---")
    pprint(raw_assignments_list[0])
    
    # OPTION 2: Use Pandas to see a readable table of just names and IDs
    print("\n--- READABLE TABLE OF ASSIGNMENTS ---")
    df_assignments = pd.DataFrame(raw_assignments_list)
    # Display only useful columns for quick scanning
    display(df_assignments[['id', 'name', 'workflow_state', 'due_at']].head(10))

-> Requesting: https://canvas.asu.edu/api/v1/courses/240102/assignments
-> Success!
Found 8 assignments.

--- RAW JSON OF FIRST ASSIGNMENT ---
{'allowed_attempts': -1,
 'annotatable_attachment_id': None,
 'anonymize_students': False,
 'anonymous_grading': False,
 'anonymous_instructor_annotations': False,
 'anonymous_peer_reviews': False,
 'assignment_group_id': 1191871,
 'automatic_peer_reviews': False,
 'can_duplicate': True,
 'course_id': 240102,
 'created_at': '2025-10-09T19:50:31Z',
 'description': '<p>qwerqwerqwerqw</p>',
 'due_at': None,
 'due_date_required': False,
 'final_grader_id': None,
 'grade_group_students_individually': False,
 'graded_submissions_exist': False,
 'grader_comments_visible_to_graders': True,
 'grader_count': 0,
 'grader_names_visible_to_final_grader': True,
 'graders_anonymous_to_graders': False,
 'grading_standard_id': None,
 'grading_type': 'points',
 'group_category_id': None,
 'has_overrides': False,
 'has_submitted_submissions': False,
 'hide_in_grad

,id,name,workflow_state,due_at
0,6781491,Test,published,None
1,6789676,Team2_Test1,published,2025-10-13T06:59:59Z
2,6801834,Operating Systems & CUDA Quiz,published,2025-10-18T06:59:00Z
3,6801843,What is CUDA and the Kernel? How Do They Relate?,published,2025-10-18T06:59:00Z
4,6801889,Roll Call Attendance,published,None
5,6803004,OS Assignment 1,published,2025-11-02T06:59:59Z
6,6803017,OS Assignment 2,published,2025-11-02T06:59:59Z
7,6803030,OS Assignment 3,published,2025-11-02T06:59:59Z


In [16]:
# REPLACE WITH AN ID FROM THE PREVIOUS CELL
TEST_ASSIGNMENT_ID = "6803004" 

endpoint = f"courses/{COURSE_ID}/assignments/{TEST_ASSIGNMENT_ID}"

# RAW API TRICK: specific 'include' parameters are often needed to get full data.
# We are asking specifically for the full rubric data to be embedded in the response.
# in canvasapi wrapper this often happens automatically, but here we must ask for it.
params = {
    "include[]": ["rubric_assessment", "submission", "score_statistics"] 
}

raw_assignment_detail = make_canvas_get_request(endpoint, params=params)

if raw_assignment_detail:
    print(f"\n--- DETAILS FOR: {raw_assignment_detail.get('name')} ---")
    
    print("\n[DESCRIPTION HTML SNIPPET]:")
    # Show first 200 chars of description to see raw HTML format
    print(str(raw_assignment_detail.get('description'))[:200] + "...")
    
    print("\n[RUBRIC RAW DATA]:")
    # Check if rubric is present in the raw response
    if 'rubric' in raw_assignment_detail:
        pprint(raw_assignment_detail['rubric'])
    else:
        print("NO RUBRIC FOUND IN RAW RESPONSE. (Did you add 'include[]' params?)")

-> Requesting: https://canvas.asu.edu/api/v1/courses/240102/assignments/6803004
-> Success!

--- DETAILS FOR: OS Assignment 1 ---

[DESCRIPTION HTML SNIPPET]:
<p>Follow all instructions and <span>submit your completed assignment as a single PDF file.</span></p>
<p><a class="instructure_file_link instructure_scribd_file inline_disabled" title="Operating Syst...

[RUBRIC RAW DATA]:
[{'criterion_use_range': False,
  'description': 'Code',
  'id': '_1268',
  'long_description': 'All code snippets are complete and correctly perform '
                      'the  appropriate task.',
  'points': 5.0,
  'ratings': [{'description': 'Full Marks',
               'id': 'blank',
               'long_description': '',
               'points': 5.0},
              {'description': 'No Marks',
               'id': 'blank_2',
               'long_description': '',
               'points': 0.0}]},
 {'criterion_use_range': False,
  'description': 'Questions',
  'id': '_6077',
  'long_description': 'All ques

In [17]:
# We use the same TEST_ASSIGNMENT_ID from Cell 6
endpoint = f"courses/{COURSE_ID}/assignments/{TEST_ASSIGNMENT_ID}/submissions"

params = {
    "per_page": 100,             # Get lots of submissions
    "include[]": ["user"],       # Include student info (names) used for testing
}

raw_submissions = make_canvas_get_request(endpoint, params=params)

if raw_submissions:
    print(f"Found {len(raw_submissions)} submissions.")
    
    # Use pandas to quickly visualize grades
    df_subs = pd.DataFrame(raw_submissions)
    
    # Check if these columns exist before trying to display them to avoid errors
    cols_to_display = ['user_id', 'score', 'grade', 'workflow_state', 'submitted_at']
    existing_cols = [col for col in cols_to_display if col in df_subs.columns]

    print("\n--- SUBMISSION SCORES TABLE ---")
    display(df_subs[existing_cols].sort_values(by='score', ascending=False).head(10))

    # Show raw JSON of the highest scoring one to see how to get attachments
    print("\n--- RAW JSON OF HIGHEST SCORING SUBMISSION ---")
    # Sort by score descending and take the first one
    highest_sub = sorted([s for s in raw_submissions if s.get('score') is not None], 
                         key=lambda x: x['score'], reverse=True)[0]
    pprint(highest_sub)

-> Requesting: https://canvas.asu.edu/api/v1/courses/240102/assignments/6803004/submissions
-> Success!
Found 1 submissions.

--- SUBMISSION SCORES TABLE ---


,user_id,score,grade,workflow_state,submitted_at
0,1445445,18.0,18,graded,2025-10-19T19:44:41Z



--- RAW JSON OF HIGHEST SCORING SUBMISSION ---
{'assignment_id': 6803004,
 'attachments': [{'category': 'uncategorized',
                  'content-type': 'application/pdf',
                  'created_at': '2025-10-19T19:44:41Z',
                  'display_name': 'ID # 1225321968 – Written Homework Week '
                                  '#1.pdf',
                  'filename': 'ID+%23+1225321968+%E2%80%93+Written+Homework+Week+%231.pdf',
                  'folder_id': 14029003,
                  'hidden': False,
                  'hidden_for_user': False,
                  'id': 118138829,
                  'lock_at': None,
                  'locked': False,
                  'locked_for_user': False,
                  'media_entry_id': None,
                  'mime_class': 'pdf',
                  'modified_at': '2025-10-19T19:44:41Z',
                  'preview_url': '/api/v1/canvadoc_session?blob=%7B%22moderated_grading_allow_list%22:null,%22enable_annotations%22:true,%22enrollmen